In [ ]:
!pip install geemap

     |████████████████████████████████| 493 kB 7.2 MB/s 
     |████████████████████████████████| 8.6 MB 36.4 MB/s 
     |████████████████████████████████| 1.6 MB 62.3 MB/s 
     |████████████████████████████████| 130 kB 50.4 MB/s 
     |████████████████████████████████| 219 kB 66.3 MB/s 
     |████████████████████████████████| 98 kB 10.1 MB/s 
     |████████████████████████████████| 1.3 MB 44.2 MB/s 
     |████████████████████████████████| 93 kB 1.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3 MB 72.0 MB/s 
     |████████████████████████████████| 76 kB 7.6 MB/s 
     |████████████████████████████████| 216 kB 75.3 MB/s 
     |████████████████████████████████| 93 kB 1.8 MB/s 
     |████████████████████████████████| 1.2 MB 80.5 MB/s 
     |████████████████████████████████| 97 kB 9.1 MB/s 
     |████████████████████████████████| 394 kB 63.9 MB/s 
     |████████

In [ ]:
import ee
import geemap
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=Q6XiK8thJK7TUhA09iuGeYqFe9gjfnJGcxxyxpGphDg&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgPX8Qzyjb6KAYnizKw3KYkLDTDVBg6vjwR61hSMKnjWeSzbhdoT5s

Successfully saved authorization token.


In [ ]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [ ]:
point = ee.Geometry.Point([80.3288, 23.5121])
# point = ee.Geometry.Point([-87.7719, 41.8799])

image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterDate('2016-01-01', '2016-12-31') \
    .sort('CLOUD_COVER') \
    .min() \
    .select('B[1-7]')

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(point, 5)
Map.addLayer(image, vis_params, "Landsat-8")

In [ ]:
map2 = geemap.Map()

ee_class_table = """

Value	Color	Description
0	1c0dff	Water
1	05450a	Evergreen needleleaf forest
2	086a10	Evergreen broadleaf forest
3	54a708	Deciduous needleleaf forest
4	78d203	Deciduous broadleaf forest
5	009900	Mixed forest
6	c6b044	Closed shrublands
7	dcd159	Open shrublands
8	dade48	Woody savannas
9	fbff13	Savannas
10	b6ff05	Grasslands
11	27ff87	Permanent wetlands
12	c24f44	Croplands
13	a5a5a5	Urban and built-up
14	ff6d4c	Cropland/natural vegetation mosaic
15	69fff8	Snow and ice
16	f9ffa4	Barren or sparsely vegetated
254	ffffff	Unclassified

"""

landcover = ee.ImageCollection('MODIS/006/MCD12Q1') \
              .filterDate('2016-01-01', '2016-12-31') \
              .select('LC_Type1')

LandCoverVis = {
  'min': 1.0,
  'max': 17.0,
  'palette': [
    '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff'
  ],
}

map2.centerObject(point, 5)
map2.addLayer(landcover, LandCoverVis, 'MODIS Land Cover')

legend_dict = geemap.legend_from_ee(ee_class_table)
map2.add_legend(legend_title="MODIS Global Land Cover", legend_dict=legend_dict)

#map2

In [ ]:
modis = ee.ImageCollection('MODIS/006/MCD12Q1') \
              .filterDate('2016-01-01', '2016-12-31') \
              .select('LC_Type1') \
              .reduce(ee.Reducer.median())

points = modis.sample(**{
    'region': ee.Geometry.Rectangle([68.0000, 37.0000, 98.0000, 6.0000]),
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(points, {}, 'training', False)
Map

Map(center=[23.512099999999997, 80.3288], controls=(WidgetControl(options=['position', 'transparent_bg'], widg…

In [ ]:
print(points.size().getInfo())

4930


In [ ]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [93.94222986465215, 18.244383326707315]}, 'id': '0', 'properties': {'LC_Type1_median': 17}}


In [ ]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# This property of the table stores the land cover labels.
label = 'LC_Type1_median'
input = image.select(bands)

# Overlay the points on the imagery to get training.
training = input.sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

print(training.first().getInfo());

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 135, 'B2': 157, 'B3': 99, 'B4': 35, 'B5': 21, 'B6': 33, 'B7': 22, 'LC_Type1_median': 17}}


In [ ]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(trained)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classfied')
Map

Map(center=[23.512099999999997, 80.3288], controls=(WidgetControl(options=['position', 'transparent_bg'], widg…

In [ ]:
landsatcover = result.set('classification_class_values', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])
landsatcover = landsatcover.set('classification_class_palette', ['1c0dff', '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159', 'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c', '69fff8', 'f9ffa4',   ])

In [ ]:
map2.addLayer(landsatcover, {}, 'Land cover')
map2

Map(center=[23.512099999999997, 80.3288], controls=(WidgetControl(options=['position', 'transparent_bg'], widg…